In [1]:
import networkx as nx
import numpy as np

In [2]:
G = nx.Graph()

for i in range(5):
    G.add_node(i, coords=np.array([i, i*2, i*3]))

In [3]:
for node, data in G.nodes.data():
    print(f"Node {node} has coordinates {data['coords']}")

Node 0 has coordinates [0 0 0]
Node 1 has coordinates [1 2 3]
Node 2 has coordinates [2 4 6]
Node 3 has coordinates [3 6 9]
Node 4 has coordinates [ 4  8 12]


In [21]:
G.add_edge(3,1)

In [23]:
G.edges.data()
G.add_edge(1,3)
G.edges.data()

EdgeDataView([(1, 3, {})])

In [ ]:
G.nodes[0]

AttributeError: 'dict' object has no attribute 'coords'

In [1]:
from functions import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1.pcd")

pcd = load_point_cloud(pcd_path)

voxel_grid, transformation_matrix = rasterize_point_cloud(pcd, voxel_size=0.1)
#visualize(VisObjVoxel(voxel_grid, transformation_matrix))
# fill holes
voxel_grid_flooded = flood_voxel_grid(voxel_grid, direction='up')
#visualize_voxel_grid(voxel_grid_flooded, transformation_matrix, invert=False)

# fill holes from top
voxel_grid_flooded = flood_voxel_grid(voxel_grid_flooded, direction='down')
#visualize_voxel_grid(voxel_grid_flooded, transformation_matrix, invert=False)

voxel_grid_inverted = invert_voxel_grid(voxel_grid_flooded)
#visualize_voxel_grid(voxel_grid_inverted, transformation_matrix, invert=False)

# skeletonize
skeleton = skeletonize_voxel_grid(voxel_grid_flooded, dilation_size=2)
#visualize([VisObjVoxel(skeleton, transformation_matrix), VisObjVoxel(voxel_grid_flooded, transformation_matrix), VisObjPCD(pcd)])

skeleton_graph = get_graph_from_voxel(skeleton, transformation_matrix, neighborhood="N26")
#visualize(VisObjGraph(skeleton_graph))

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd...
Loaded point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd with 44026810 points.
Rasterizing point cloud with voxel size 0.1...
Voxel grid created with shape (248, 484, 54)
Filling voxel grid from direction up...
Voxel grid filled from direction up.
Filling voxel grid from direction down...
Voxel grid filled from direction down.
Inverting voxel grid...
Voxel grid inverted.
Skeleton shape: (250, 486, 56)
Created skeleton with 11457 voxels.
Converting voxel grid to graph...
Total nodes added: 11457
Adding edges...
Processing node 0/11457
Processing node 1/11457
Processing node 1001/11457
Processing node 2001/11457
Processing node 3001/11457
Processing node 4001/11457
Processing node 5001/11457
Processing node 6001/11457
Processing node 7001/11457
Processing node 8001/11457
Processing node 9001/11457
Processing node 10001/11457
Processing node 11001/11457
Created graph with 11457

In [26]:
counter = 0
for node, data in skeleton_graph.nodes.data():
    #print(f"Node {node} has data {data}")
    if 'coords' not in data:
        print(f"Node {node} does not have 'coords' attribute. Other attributes: {data}")
        counter += 1
print(f"Total nodes without 'coords': {counter}")
print(skeleton_graph.number_of_nodes())


Node (np.int64(9), np.int64(457), np.int64(27)) does not have 'coords' attribute. Other attributes: {}
Node (np.int64(9), np.int64(458), np.int64(26)) does not have 'coords' attribute. Other attributes: {}
Node (np.int64(9), np.int64(459), np.int64(25)) does not have 'coords' attribute. Other attributes: {}
Node (np.int64(9), np.int64(460), np.int64(25)) does not have 'coords' attribute. Other attributes: {}
Node (np.int64(9), np.int64(461), np.int64(24)) does not have 'coords' attribute. Other attributes: {}
Node (np.int64(10), np.int64(456), np.int64(27)) does not have 'coords' attribute. Other attributes: {}
Node (np.int64(11), np.int64(456), np.int64(27)) does not have 'coords' attribute. Other attributes: {}
Node (np.int64(12), np.int64(456), np.int64(28)) does not have 'coords' attribute. Other attributes: {}
Node (np.int64(13), np.int64(455), np.int64(28)) does not have 'coords' attribute. Other attributes: {}
Node (np.int64(14), np.int64(455), np.int64(28)) does not have 'coord

In [29]:
coordinates = np.random.rand(10, 3)

G = nx.Graph()
for i, coords in enumerate(coordinates):
    G.add_node(i, coords=coords)

In [5]:
np.unique(voxel_grid)

array([0, 1])

In [31]:
x = {1: 'test', 5: 'demo'}

In [33]:
x[1]

'test'

In [6]:
import numpy as np
from pathlib import Path

data = np.load(Path("../MSSP/output/ske_ids.npy"))

print(data.dtype)

int32


In [9]:
import numpy as np
arr = np.load("../MSSP/input/M.npy", allow_pickle=False)
print(arr.shape)
print(arr.nbytes)
print(arr.size)
print(arr.dtype)

(4, 4)
128
16
float64
